In [209]:
import leveldb
import gzip
import re
import codecs
import json
from tqdm import tqdm
from pymongo import MongoClient
import pymongo
from flask import Flask, render_template, request

In [133]:
leveldb.LevelDB("./music.ldb")

LevelDBError: IO error: lock ./music.ldb/LOCK: already held by process

In [135]:
def nlp60(db):
    with codecs.open("./artist.json", "r", encoding="utf-8") as f:
        for i in tqdm(f):
            data = json.loads(i)
            db.put(data["name"].encode("utf-8"), data.get("area","").encode("utf-8"))
if __name__ == "__main__":
    my_db = plyvel.DB("./name_area.ldb", create_if_missing=True)
    nlp60(my_db)
    my_db.close()     


0it [00:00, ?it/s]
3300it [00:00, 32562.33it/s]
6995it [00:00, 34788.21it/s]
10952it [00:00, 36315.78it/s]
14037it [00:00, 34955.13it/s]
17465it [00:00, 34792.63it/s]
20948it [00:00, 34832.41it/s]
24632it [00:00, 35081.96it/s]
27878it [00:00, 34154.84it/s]
31470it [00:00, 34339.85it/s]
35066it [00:01, 34476.00it/s]
38427it [00:01, 34122.90it/s]
41902it [00:01, 34171.01it/s]
45700it [00:01, 34455.65it/s]
49395it [00:01, 34628.80it/s]
53038it [00:01, 34752.38it/s]
56622it [00:01, 34805.03it/s]
60197it [00:01, 34774.69it/s]
63732it [00:01, 34720.04it/s]
67210it [00:01, 34643.42it/s]
70773it [00:02, 34698.09it/s]
74450it [00:02, 34786.57it/s]
78102it [00:02, 34865.05it/s]
81672it [00:02, 34811.22it/s]
85177it [00:02, 34780.79it/s]
88762it [00:02, 34816.83it/s]
92271it [00:02, 34810.44it/s]
95867it [00:02, 34847.39it/s]
99668it [00:02, 34956.61it/s]
103274it [00:02, 34986.81it/s]
106875it [00:03, 35011.55it/s]
110469it [00:03, 34982.36it/s]
114009it [00:03, 34969.51it/s]
117706it [00:03, 3

In [142]:
my_db.close()

In [143]:
def nlp61(name, db):
    return db.get(name.encode())

if __name__ == "__main__":
    my_db = plyvel.DB("./name_area.ldb", create_if_missing=True)
    print(nlp61("The Wanderers", my_db))
    my_db.close()

b'United States'


In [150]:
def nlp62(value, db):
    count = 0
    for k, v in db.iterator():
        if v == value.encode("utf-8"):
            count += 1
    return count

if __name__ == "__main__":
    my_db = plyvel.DB("./name_area.ldb", create_if_missing=True)
    print(nlp62("Japan", my_db))
    my_db.close()

21946


In [159]:
def nlp63(name, db, is_db_built=True):
    if not is_db_built:
        with codecs.open("./artist.json", "r", encoding="utf-8") as f:
            for i in tqdm(f):
                data = json.loads(i)
                db.put(data["name"].encode("utf-8"), 
                       json.dumps(data.get("tags","")).encode("utf-8"))
        print("finished")
    return json.loads(db.get(name.encode()).decode())
    
if __name__ == "__main__":
    my_db = plyvel.DB("./name_tags.ldb", create_if_missing=True)
    print(nlp63("WIK▲N", my_db, is_db_built=True))
    my_db.close()

[{'count': 1, 'value': 'sillyname'}]


In [ ]:
def nlp64(db):
    insert_data = []
    collection = db.musician_field
    with codecs.open("./artist.json", "r", encoding="utf-8") as f:
            for i in tqdm(f):
                data = json.loads(i)
                """
                name = data.get("name", "")
                aliases_name = "" if data.get("aliases") is None \
                                else data["aliases"]["name"]
                tag = "" if data.get("tags") is None \
                                else data["tags"]["value"]
                rating = "" if data.get("rating") is None \
                            else data["rating"]["value"]
                new_data = {u"name":name.encode(),
                           u"aliases_name" : aliases_name.encode(),
                           u"tag" : tag.encode(),
                           u"rating" : rating.encode()}
                """
                insert_data.append(data)
    result = collection.insert_many(insert_data)
    collection.create_index([('name', pymongo.ASCENDING)])  
    collection.create_index([('aliases.name', pymongo.ASCENDING)])  
    collection.create_index([('tags.value', pymongo.ASCENDING)])
    collection.create_index([('rating.value', pymongo.ASCENDING)])
    print("finised")
    
if __name__ == "__main__":
    client = MongoClient("localhost",27017)
    nlp64(client.nlp100)

In [171]:
def nlp65(db):
    queen_info = []
    collection = db.musician_field
    for queen in collection.find({"name":"Queen"}):
        queen_info.append(queen)
    return queen_info
        
if __name__ == "__main__":
    client = MongoClient("localhost",27017)
    print(nlp65(client.nlp100))

[{'_id': ObjectId('5b4c33708bb95b3389349e0d'), 'name': 'Queen', 'area': 'Japan', 'gender': 'Female', 'tags': [{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}], 'sort_name': 'Queen', 'ended': True, 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7', 'type': 'Character', 'id': 701492, 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}, {'_id': ObjectId('5b4c33718bb95b33893564b9'), 'rating': {'count': 24, 'value': 92}, 'begin': {'date': 27, 'month': 6, 'year': 1970}, 'name': 'Queen', 'area': 'United Kingdom', 'tags': [{'count': 2, 'value': 'hard rock'}, {'count': 1, 'value': '70s'}, {'count': 1, 'value': 'queen family'}, {'count': 1, 'value': '90s'}, {'count': 1, 'value': '80s'}, {'count': 1, 'value': 'glam rock'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'english'}, {'count': 2, 'value': 'uk'}, {'count': 1, 'value': 'pop/rock'}, {'count': 1, 'value': 'pop-rock'}, {'count': 1, 'value': 'britannique'}, {'count': 1, 'value': 'classic pop and rock'

In [180]:
def nlp67(db, name):
    result = []
    collection = db.musician_field
    for i in tqdm(collection.find({"aliases.name": name})):
        result.append(i["name"])
    return list(set(result))

if __name__ == "__main__":
    client = MongoClient("localhost",27017)
    print(nlp67(client.nlp100, "女王"))

2it [00:00, 844.26it/s]

['Queen']


In [200]:
def nlp68(db, tag_name):
    count = 0
    result = []
    collection = db.musician_field
    for i, j in enumerate(
        collection.find({"tags.value" : tag_name}).sort(
            "rating.count",-1)):
        if count >= 10:
            break
        if j["name"] not in result:
            result.append(j["name"])
            count += 1
    return result

if __name__ == "__main__":
    client = MongoClient("localhost",27017)
    print(nlp68(client.nlp100, "dance"))

['Madonna', 'Björk', 'The Prodigy', 'Rihanna', 'Britney Spears', 'Maroon 5', 'Adam Lambert', 'Fatboy Slim', 'Basement Jaxx', 'Cornershop']


In [252]:
def search(db, name, aliases_name, tag):
    result = []
    count = 0
    name_ls = []
    collection = db.musician_field
    search_indexes = ["name","aliases.name","tags.value"]
    search_query = {i : j for i, j in zip(search_indexes, [name, aliases_name, tag])
                   if len(j) > 0}
    for value in collection.find(search_query).sort("rating.value", -1):
        if value.get("name") in name_ls:
            continue
        name_ls.append(value.get("name",""))
        result.append(",".join([value.get("name",""), value.get("area", ""),
                      value.get("gender",""), value.get("aliases.name","")]))
        count += 1
        if count == 9:
            break
    return result

In [258]:
def nlp69():
    app = Flask(__name__)
    @app.route("/")
    def index():
        return render_template("index.html")

    @app.route('/post', methods=['GET', 'POST'])
    def post():
        if request.method == "POST":
            name = request.form["name"]
            aliases_name = request.form["aliases_name"]
            tag = request.form["tag.value"]
            result = search(client.nlp100, name, aliases_name, tag)
        return render_template("index.html", result = result)
    app.run()
    
if __name__ == "__main__":
    nlp69()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2018 00:38:22] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 00:38:32] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2018 00:38:40] "POST /post HTTP/1.1" 200 -


In [253]:
search(client.nlp100, "Queen", "", "hard rock")

['Queen,United Kingdom,,']